# Foobar Challenge lvl 3



## Problem 1

**Making fuel for the LAMBCHOP's reactor core is a tricky process because of the exotic matter involved. It starts as raw ore, then during processing, begins randomly changing between forms, eventually reaching a stable form. There may be multiple stable forms that a sample could ultimately reach, not all of which are useful as fuel.**

**Commander Lambda has tasked you to help the scientists increase fuel creation efficiency by predicting the end state of a given ore sample. You have carefully studied the different structures that the ore can take and which transitions it undergoes. It appears that, while random, the probability of each structure transforming is fixed. That is, each time the ore is in 1 state, it has the same probabilities of entering the next state (which might be the same state).  You have recorded the observed transitions in a matrix. The others in the lab have hypothesized more exotic forms that the ore can become, but you haven't seen all of them.**

**Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is guaranteed that no matter which state the ore is in, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The ore starts in state 0. The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly.**

For example, consider the matrix m:
- [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
- [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
- [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
- [0,0,0,0,0,0],  # s3 is terminal
- [0,0,0,0,0,0],  # s4 is terminal 
- [0,0,0,0,0,0],  # s5 is terminal
  
  
So, we can consider different paths to terminal states, such as:
* s0 -> s1 -> s3
* s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4
* s0 -> s1 -> s0 -> s5

Tracing the probabilities of each, we find that
* s2 has probability 0
* s3 has probability 3/14
* s4 has probability 1/7
* s5 has probability 9/14

So, putting that together, and making a common denominator, gives an answer in the form of
[s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator] which is
[0, 3, 2, 9, 14].

### The first approach

**Both approach are the same, but this works only in Python 3 because this version support numpy and all the functions for matrices**

In [ ]:
import numpy as np
import fractions

def solution(m):
    
    # Matrix_no_terminal is only the non-Terminal states, in order to find the fundamental matrix of the absorbing Markov Chain
    # Q_matrix and R matrix corresponds to the slice of the non-terminal states if the form:
    #                           Absorbing   non-absorbing
    #        P = absorbing      (    I         0      )
    #            non-absorbing  (    R         Q      )
    #
    # Terminal and no_terminal arrays are the indexes where we find the terminal and non-terminal states in each row
    # FR gives the matrix of probabilities that a particular initial non-absorbing state will lead to a particular absorbing state
    matrix_no_terminal = []
    Q_matrix = []
    R_matrix = []
    terminal = []
    no_terminal = []
    FR = []

    # The original input give us the occurrences from state to another state, we need the probability gice a solution
    # for this, we sum the entire row and divide each number in a row by it self sum.
    def fix_input(m):
        temp_matrix = []
        
        for row in m:
            temp_array = []
            sum_row = sum(row)
            
            if sum_row != 0:
                for num in row:
                    temp_array.append(num/sum_row)
            else:
                temp_array = row
            
            temp_matrix.append(temp_array)
        
        return temp_matrix
    
    # This function recognize the terminal states seraching if all the numbers in the state are 0
    # and the non-Terminal states and append it to their respective list.
    def is_terminal(m):
        i = 0 
        
        for row in m:
            
            if (all(x == 0 for x in row)):
                terminal.append(i)
            else:
                no_terminal.append(i)
            i += 1
            
        return terminal, no_terminal
    
    # In order to adquire R and Q matrix we need to sort in the non-terminal states, first the indexes corresponding to terminal
    # state and then the non-terminal state.
    def sort_states(state, index_terminal, index_Noterminal): 
        new_state =[]

        for index in index_terminal:
            new_state.append(state[index])
        
        for index in index_Noterminal:
            new_state.append(state[index])
        
        return new_state
    
    # Matrix FR is calculated as Identity Matrix - Q_matrix, then calculating the invers for this matrix and finally calculating
    # the dot product between F matrix and R matrix.
    def matrix_FR (R_matrix, Q_matrix):
        
        R_matrix = np.array(R_matrix)
        Q_matrix = np.array(Q_matrix)
        x, y = np.shape(Q_matrix)
        Ident_matrix = np.identity(x) 
        F = np.subtract(Ident_matrix,Q_matrix)
        F = np.linalg.inv(F)
        
        return np.dot(F, R_matrix)

    # This function have as parameter an array of probabilities from state 0 to each terminal state, extract the numerators and 
    # denominators, calculate the lcm for the denominators and return an array in the form [int terminal S1, int terminal S2,..., int denominator for each probability]
    def numerators(array_fractions):
        
        denominators = []
        numerators = [] 
        for num in array_fractions:
            
            if '/' in num:
                denominators.append(int(num.split('/')[1]))
                numerators.append(int(num.split('/')[0]))
            else:
                denominators.append(1)
                numerators.append(int(num.split('/')[0]))

        denominators = np.array(denominators)
        lcm = np.lcm.reduce(denominators)

        i = 0
        while i < len(numerators):
            numerators[i] = numerators[i] * int(lcm/denominators[i])
            i+=1
        numerators.append(lcm)
        return numerators
    
    
    m = fix_input(m)    
    
    # Adquire the indexes for terminal and non-terminal states from the matrix m
    terminal, no_terminal = is_terminal(m)
    
    # Iterate through the non terminal indexes and store in temp var the sorted row of non terminal in the form: 
    # [terminal states, nonterminal state] and create a matrix with all the rows of non terminal states
    for state in no_terminal:
        temp = sort_states(m[state], terminal, no_terminal)
        matrix_no_terminal.append(temp)
    
    # We need to create two different matrices, the first is the terminals state from the matrix of non-terminal states,
    # and the second the non terminals values from the non terminal states
    for state in matrix_no_terminal:
        R_matrix.append(state[:len(terminal)])
        Q_matrix.append(state[len(terminal):])
        
    # This set print is to print as fraction the numbers in the solution matrix
    np.set_printoptions(formatter={'all':lambda x: str(fractions.Fraction(x).limit_denominator())})
    FR = matrix_FR (R_matrix,Q_matrix)
    
    # Take the state cero array and convert it to an array of strings in order to extract the numerators and denominators
    state_cero_start = (str(FR[0])[1:-1]).split(' ')

    solution = numerators(state_cero_start)
    return numerators(state_cero_start)

## The second approach

**Its the same approach, absorbing markov chain, but all the matrices operations are made manually**

In [5]:
from fractions import Fraction


def solution(m):
    """
     Matrix_no_terminal is only the non-Terminal states, in order to find the fundamental matrix of the absorbing
     Markov Chain Q_matrix and R_matrix corresponds to the slice of the non-terminal states if the form:
                                        Absorbing  non-absorbing
                    P = absorbing      (    I         0      )
                        non-absorbing  (    R         Q      )

     Terminal and no_terminal arrays are the indexes where we find the terminal and non-terminal states in each row;
     FR gives the matrix of probabilities that a particular initial non-absorbing state will lead to a particular
     absorbing state"""

    matrix_no_terminal = []
    Q_matrix = []
    R_matrix = []
    terminal = []
    no_terminal = []
    FR = []

    # The original input give us the occurrences from state to another state, we need the probability to give a solution
    # for this, we sum the entire row and divide each number in a row by it self row sum.
    def fix_input(m_input):
        temp_matrix = []

        for row in m_input:
            temp_array = []
            sum_row = sum(row)

            if sum_row != 0:
                for number in row:
                    if number != 0:
                        temp_array.append(Fraction(number, sum_row))
                    else:
                        temp_array.append(number)
            else:
                temp_array = row

            temp_matrix.append(temp_array)

        return temp_matrix

    # Iterate through row and index to substract x[row][column] - y[row][column]
    def substract_matrix(x_in, y_in):

        rows_substract = len(x_in)
        columns_substract = len(x_in[0])
        sol_matrix_substract = []

        row = 0
        while row < rows_substract:

            sol_array_substract = [0] * columns_substract
            col = 0

            while col < columns_substract:
                sol_array_substract[col] = x_in[row][col] - y_in[row][col]
                col += 1

            sol_matrix_substract.append(sol_array_substract)
            row += 1

        return sol_matrix_substract

    # This function creates the identity matrix n by n, to use it in the calculations
    def identity_matrix(reference_matrix):
        identity_matrix_created = []
        n = len(reference_matrix)

        for index in range(n):
            row = [Fraction(0, 1)] * n
            row[index] = Fraction(1, 1)
            identity_matrix_created.append(row)

        return identity_matrix_created

    # This function calculate the dot product between the matrices, multiplying the row of x by the column of y
    def dot_product(x_dot, y_dot):

        columns_dot = len(y_dot[0])
        dot_matrix_created = []

        # iterating by row of X
        for i_dot in range(len(x_dot)):
            row = [0] * columns_dot

            # iterating by column by y
            for j_dot in range(len(y_dot[0])):

                # iterating by index of row array, and adding the multiplication
                for k_dot in range(len(y_dot)):
                    row[j_dot] += (x_dot[i_dot][k_dot] * y_dot[k_dot][j_dot])

            dot_matrix_created.append(row)

        return dot_matrix_created

    # This function recognize the terminal states searching if all the numbers in the state are 0 or 1 in de specific
    # index and the non-Terminal states and append it to their respective list.
    def is_terminal(m_input):
        i_terminal_row = 0
        identify_terminal = []
        identify_no_terminal = []

        for row in m_input:
            if sum(row) == 0 or (sum(row) == 1 and m_input[i_terminal_row][i_terminal_row] == 1):
                identify_terminal.append(i_terminal_row)
            else:
                identify_no_terminal.append(i_terminal_row)
            i_terminal_row += 1

        return identify_terminal, identify_no_terminal

    # In order to acquire R and Q matrix we need to sort in the non-terminal states, first the indices corresponding
    # to the terminal states and then the non-terminal states.
    def sort_states(state_in, index_terminal, index_no_terminal):
        new_state = []

        for index in index_terminal:
            new_state.append(state_in[index])

        for index in index_no_terminal:
            new_state.append(state_in[index])

        return new_state

    # This function create a matrix of zeros n by n
    def zeros_matrix(rows, cols):
        created_matrix = []
        for i in range(rows):
            created_matrix.append([])
            for j in range(cols):
                created_matrix[-1].append(0.0)

        return created_matrix

    # This function creates an exact copy of the input matrix, and return that copy matrix
    def copy_matrix(matrix_copy):
        rows = len(matrix_copy)
        cols = len(matrix_copy[0])

        MC = zeros_matrix(rows, cols)

        for i in range(rows):
            for j in range(rows):
                MC[i][j] = matrix_copy[i][j]

        return MC


    def inverse_matrix(matrix_in):
        n = len(matrix_in)
        AM = copy_matrix(matrix_in)
        IM = identity_matrix(matrix_in)

        # Perform row operations
        indices = list(range(n))
        for fd in range(n):  # fd stands for focus diagonal
            fdScaler = Fraction(1, AM[fd][fd])
            # FIRST: scale fd row with fd inverse.
            for j in range(n):  # Use j to indicate column looping.
                AM[fd][j] *= fdScaler
                IM[fd][j] *= fdScaler

            # SECOND: operate on all rows except fd row as follows:
            for i in indices[0:fd] + indices[fd + 1:]:
                crScaler = AM[i][fd]
                for j in range(n):
                    # cr - crScaler * fdRow, but one element at a time.
                    AM[i][j] = AM[i][j] - crScaler * AM[fd][j]
                    IM[i][j] = IM[i][j] - crScaler * IM[fd][j]
        return IM

    # Matrix FR is calculated as Identity Matrix - Q_matrix, then calculating the invers for this matrix and finally
    # calculating the dot product between F matrix and R matrix.
    def matrix_fr_calculation(r_matrix_in, q_matrix_in):

        indent_matrix = identity_matrix(q_matrix_in)
        f_matrix = substract_matrix(indent_matrix, q_matrix_in)
        f_matrix = inverse_matrix(f_matrix)

        return dot_product(f_matrix, r_matrix_in)

    # This function have as parameter an array of probabilities from state 0 to each terminal state, extract the
    # numerators and denominators, calculate the lcm for the denominators and return an array in the form [int
    # terminal S1, int terminal S2,..., int denominator for each probability]

    # Checks if num is prime
    def is_prime(number_prime):
        # 1 is not a prime, however, for purposes of the algorithm
        # it's convenient to assume the contrary
        if number_prime == 2 or number_prime == 1:
            return True

        if not (number_prime % 2):
            return False

        count = 3
        while count <= (number_prime ** 0.5):
            if not (number_prime % count):
                return False
            count += 2

        return True

    # Takes integer and returns list with prime decomposition
    def prime_decom(number_decom):
        primes = []

        while number_decom >= 1:

            if is_prime(number_decom):
                primes.append(number_decom)
                return primes

            count_decom = 2
            while count_decom <= (number_decom ** 0.5):

                if not (number_decom % count_decom):
                    primes.append(count_decom)
                    number_decom = number_decom / count_decom
                count_decom += 1

    # takes list of lists and returns list with unique values
    def unique_values(my_list):
        uniqueValues = []

        for currList in my_list:
            for number in currList:
                if number not in uniqueValues:
                    uniqueValues.append(number)

        return uniqueValues

    '''
    Takes list of prime decompositions and unique values
    and returns Least Common Multiple list in the form (x,y) 
    where x is the prime and y the exponent. 
    '''

    def lcm_list(prime_decom_in, unique_values_in):
        # list of lists, each with prime and its exponent
        maxValue = []

        for value in unique_values_in:
            maxAmnt = 0

            for currList in prime_decom_in:
                currMax = 0

                for number in currList:
                    if number == value:
                        currMax += 1
                if currMax > maxAmnt:
                    maxAmnt = currMax

            maxValue.append([value, maxAmnt])

        return maxValue

    # Computes multiplication of values on Least Common Multiple list
    def lcm(lcm_list_in):
        lcm_curr = 1
        for currValue in lcm_list_in:
            lcm_curr = lcm_curr * (currValue[0] ** currValue[1])

        return int(lcm_curr)

    # Takes a list of Integers and returns their Least Common Multiple
    def lcm_from_list_of_integers(my_list):
        prime_factorizations = []

        # Take prime decomposition of each number and add it to list
        for nums in my_list:
            prime_factorizations.append(prime_decom(nums))

        '''
        Takes unique values for list of decompositions.
        This is done to later count the maximum amount of
        occurences of each prime through all decompositions.
        '''
        uniqueVal = unique_values(prime_factorizations)

        return lcm(lcm_list(prime_factorizations, uniqueVal))

    '''This is the main code, first it fix the input matrix, converting it into a probabilities matrix, second, 
    it recognize the terminal and non terminal states, third, it takes de R and Q matrices from the sort absorbing 
    markov chain matrix, fourth, it calculates FR matrix that is FR = ((In-Q)^-1) . R, and finally it obtain the row 
    starting from state 0, calculate the least common multiple and give the solution. '''
    m_fixed = fix_input(m)

    # Acquire the indexes for terminal and non-terminal states from the matrix m
    terminal, no_terminal = is_terminal(m)

    if len(m_fixed) == 1 and len(m_fixed[0]) == 1:
        return [1, 1]

    # Iterate through the non terminal indexes and store in temp var the sorted row of non terminal in the form:
    # [terminal states, no_terminal state] and create a matrix with all the rows of non terminal states
    for state in no_terminal:
        temp = sort_states(m_fixed[state], terminal, no_terminal)
        matrix_no_terminal.append(temp)

    # We need to create two different matrices, the first is the terminals state from the matrix of non-terminal states,
    # and the second the non terminals values from the non terminal states
    for state in matrix_no_terminal:
        R_matrix.append(state[:len(terminal)])
        Q_matrix.append(state[len(terminal):])

    FR = matrix_fr_calculation(R_matrix, Q_matrix)
    # Take the state Zero array and convert it to an array of strings in order to extract the numerators and
    # denominators
    state_Zero_start = FR[0]
    denominators = []

    for num in state_Zero_start:
        denominators.append(num.denominator)

    lcm_var = lcm_from_list_of_integers(denominators)
    results = []

    state_Zero_start.append(lcm_var)

    i = 0
    while i < (len(state_Zero_start) - 1):
        state_Zero_start[i] = (state_Zero_start[i] * state_Zero_start[-1])
        i += 1

    for num in state_Zero_start[:-1]:
        results.append(num.numerator)

    results.append(state_Zero_start[-1])

    return results


[0, 3, 2, 9, 14]

## Problem 2

You're so close to destroying the LAMBCHOP doomsday device you can taste it! But in order to do so, you need to deploy special self-replicating bombs designed for you by the brightest scientists on Bunny Planet. There are two types: Mach bombs (M) and Facula bombs (F). The bombs, once released into the LAMBCHOP's inner workings, will automatically deploy to all the strategic points you've identified and destroy them at the same time. 

But there's a few catches. First, the bombs self-replicate via one of two distinct processes: 
Every Mach bomb retrieves a sync unit from a Facula bomb; for every Mach bomb, a Facula bomb is created;
Every Facula bomb spontaneously creates a Mach bomb.

For example, if you had 3 Mach bombs and 2 Facula bombs, they could either produce 3 Mach bombs and 5 Facula bombs, or 5 Mach bombs and 2 Facula bombs. The replication process can be changed each cycle. 

Second, you need to ensure that you have exactly the right number of Mach and Facula bombs to destroy the LAMBCHOP device. Too few, and the device might survive. Too many, and you might overload the mass capacitors and create a singularity at the heart of the space station - not good! 

And finally, you were only able to smuggle one of each type of bomb - one Mach, one Facula - aboard the ship when you arrived, so that's all you have to start with. (Thus it may be impossible to deploy the bombs to destroy the LAMBCHOP, but that's not going to stop you from trying!) 

You need to know how many replication cycles (generations) it will take to generate the correct amount of bombs to destroy the LAMBCHOP. Write a function solution(M, F) where M and F are the number of Mach and Facula bombs needed. Return the fewest number of generations (as a string) that need to pass before you'll have the exact number of bombs necessary to destroy the LAMBCHOP, or the string "impossible" if this can't be done! M and F will be string representations of positive integers no larger than 10^50. For example, if M = "2" and F = "1", one generation would need to pass, so the solution would be "1". However, if M = "2" and F = "4", it would not be possible.



Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --
Input:
Solution.solution('2', '1')
Output:
    1

Input:
Solution.solution('4', '7')
Output:
    4

-- Python cases --
Input:
solution.solution('4', '7')
Output:
    4

Input:
solution.solution('2', '1')
Output:
    1

In [142]:
def solution(M, F):
    steps = 0
    Mach = int(M)
    Fecula = int(F)
    
    
    while Mach!=1 and Fecula!=1:
        print('entro en while')
        if Fecula % Mach == 0 or Mach%Fecula == 0:
            return 'impossible'
            
        if Mach < Fecula:
            Fecula -= Mach
            steps += 1

        else:
            Mach -= Fecula
            steps +=1

    return str(steps+max(Mach, Fecula)-1)
       

In [176]:
def solution_2(M, F):
    Mach = int(M) 
    Facula = int(F)
    
    steps = 0
    
    while Mach != 1 and Facula != 1 :
        print(Mach, Facula)
        if Mach % Facula == 0:
            return 'impossible'
            
        else:
            steps += int(max(Mach, Facula)/min(Mach, Facula))
            Mach, Facula = max(Mach, Facula) % min(Mach, Facula), min(Mach, Facula) 
    
    print('b:', Mach, Facula)
    return str(steps + max(Mach, Facula)-1)

## Problem 3

Commander Lambda has asked for your help to refine the automatic quantum antimatter fuel injection system for the LAMBCHOP doomsday device. It's a great chance for you to get a closer look at the LAMBCHOP -- and maybe sneak in a bit of sabotage while you're at it -- so you took the job gladly. 

Quantum antimatter fuel comes in small pellets, which is convenient since the many moving parts of the LAMBCHOP each need to be fed fuel one pellet at a time. However, minions dump pellets in bulk into the fuel intake. You need to figure out the most efficient way to sort and shift the pellets down to a single pellet at a time. 

The fuel control mechanisms have three operations: 

1) Add one fuel pellet

2) Remove one fuel pellet

3) Divide the entire group of fuel pellets by 2 (due to the destructive energy released when a quantum antimatter pellet is cut in half, the safety controls will only allow this to happen if there is an even number of pellets)

Write a function called solution(n) which takes a positive integer as a string and returns the minimum number of operations needed to transform the number of pellets to 1. The fuel intake control panel can only display a number up to 309 digits long, so there won't ever be more pellets than you can express in that many digits.

For example:
solution(4) returns 2: 4 -> 2 -> 1

solution(15) returns 5: 15 -> 16 -> 8 -> 4 -> 2 -> 1


In [ ]:
def solution(m):
    
    def generate_reference_list(number):
        temp_list = []
        value = 2
        power = 2
        while number > value:
            temp_list.append(value)
            value = value ** power
            power 
            

In [49]:
def solution(m):
    
    # This function counts the number of times a unmber can be divided by 2, evenly
    def function_count_steps(number):
        count = 0
        
        while number % 2 == 0:
            number = number / 2
            count += 1
            
            if number == 1:
                return number, count
                
        return number, count
    
    
    m = int(m)
    steps = 0
    
    # While the input number is different from 1, if the number is divisible by 2, it add 1 step, if not,
    # it verify if it is more efficient add or substract 1, in case of the temp_plus or temp_subs give 1 as result
    # takes thata way and return te previous steps plus specific count and add 1 that is the step of add or substract 1
    while m != 1:

        if m%2 == 0:
            m = m/2
            steps += 1

            
        else:
            temp_plus = m + 1
            count_plus= 0
            temp_subs = m - 1
            count_subs = 0
            
            if m == 3:
                m = 1
                steps += 2
                break
                
            m, count_plus = function_count_steps(temp_plus)
            
            if m == 1:
                steps = steps + count_plus + 1
                break
                
            m, count_subs = function_count_steps(temp_subs)
            
            if m == 1:
                steps = steps + count_subs +  1
                break
            
            # This part of code evaluate if adding 1 or substracting 1 is more divisible in 2 and m takes the last value
            # of temp_plus or temp_subs and begin again the procces from this number
            if count_plus > count_subs:
                m = temp_plus
                steps += 1
                
            else:
                m = temp_subs
                steps += 1
                
    return str(steps)
            


In [52]:
solution('4')

'2'

In [10]:
32871264213481324698123498732461876429125%2

1

In [ ]:
def solution(m):
    m = int(m)
    steps = 0
    
    while m != 1:
        #print('-------------------------')
        if m%2 == 0:
            m = m/2
            steps += 1
            #print('divisible en 2:', m, steps)
            
        else:
            temp_plus = m + 1
            count_plus= 0
            temp_subs = m - 1
            count_subs = 0
            
            if m == 3:
                m = 1
                steps += 2
                break
            
            while temp_plus % 2 == 0:
                #print('-----temp_plus----------')
                temp_plus = temp_plus/2
                count_plus += 1
                    
                if temp_plus == 1:
                    steps += count_plus
                    m = 1
                #print('temp_plus:', temp_plus, count_plus)
            
            #print('salio while temp_plus')
            if m == 1:
                steps += 1
                break
            
            while temp_subs % 2 == 0:
                #print('--------------temp_subs--------')
                
                temp_subs = temp_subs / 2
                count_subs += 1
                
                if temp_subs == 1:
                    steps += count_subs
                    m = 1
                #print('temp_plus:', temp_subs, count_subs)
            #print('salio while temp_subs')        
            if m == 1:
                steps += 1
                break
            
            if count_plus > count_subs:
                m = temp_plus
                steps += count_plus + 1
                
            else:
                m = temp_subs
                steps += count_subs + 1
    return str(steps)

In [21]:
from fractions import Fraction

def set_diagonal(row, index_row):
    
    value = row[index_row]
    temp_row = []
    
    for i in row:
        temp_row.append(Fraction(i, value))
        
    return temp_row

def clean_column(matrix_in, index_column):
    
    for row in matrix_in:
        row[index_column] = (matrix_in[index_column][index_column] * row[index_column]) - row[index_column]
    
    print(matrix_in)

#print(set_diagonal([15,30,45],0))

print(clean_column([[1, 1, 1],[3, 3, 3],[8, 8, 8]], 0))
    

[[0, 1, 1], [-3, 3, 3], [-8, 8, 8]]
None
